In [1]:
import numpy as np


import warnings

warnings.filterwarnings('ignore')

In [2]:

PATH_IMAGENS = '../dataset/lamina/4_divisao_deixar_apenas_uma_imagem_por_campo_de_lamina_SEM_ERROS/Negativo/'
PATH_MASK = '../dataset/labelMe/collection/masks-geral/'
PATH_INDIVIDUAL = '../dataset/labelMe/collection/masks-individual/'


In [3]:
from data_load import load_crop_images

dataset = load_crop_images(PATH_IMAGENS,
                           PATH_MASK,
                           PATH_INDIVIDUAL) 

Total de Imagens:  72


  0%|          | 0/72 [00:00<?, ?it/s]

shape (1068, 761)


IndexError: There are only 0 images in the collection

In [ ]:
dataset.info()

In [ ]:
dataset['label']

In [ ]:
print("Quantidade de recortes gerados: ", len(dataset))  # (34.192 128x128) # (12.016 256x256)

In [ ]:
from meanshift.plots import exibe2imagens

imgs_ = np.random.choice(dataset.index, 5)
for img_ in imgs_:
    exibe2imagens((dataset.loc[img_, 'img']), dataset.loc[img_, 'mask'])

In [ ]:

from matplotlib import pyplot as plt
from meanshift import aplicar_menshift_paralelo
from multiprocessing import Pool

pool = Pool(processes=100)
imagens_meanshift2 = aplicar_menshift_paralelo(dataset['img'])
pool.close()
pool.join()

# Exibição dos resultados
plt.figure()
plt.title('Segmentação após Mean Shift e Pós-processamento')
plt.imshow(imagens_meanshift2[0], cmap='gray')
plt.show()


In [ ]:
from validator import media_dice

media_dice(imagens_meanshift2, dataset['mask'])


In [ ]:
dataset['mask'][0].shape

In [ ]:
imagens_meanshift2[0].re

In [ ]:
from skimage.io import imsave

#save_imgs
path = 'dataset/meanshift/predictions/negativo/'

#create path
import os
if not os.path.exists(path):
    os.makedirs(path)

for i, img in enumerate(imagens_meanshift2):
    plt.imsave(f'{path}/{i}.png', img, cmap='gray')
    plt.imsave(f'{path}/{i}_mask.png', dataset['mask'][i].reshape(128,128), cmap='gray')

In [ ]:
from skimage.color import rgb2gray

img_gray = rgb2gray(dataset['img'][200])


In [ ]:
img_gray

In [ ]:
plt.imshow(img_gray, cmap='gray')

In [ ]:
teste = imagens_meanshift2[200].copy()

In [ ]:
img_gray.dtype
teste = teste.astype(np.float64)

In [ ]:
img_final = teste * img_gray

In [ ]:
img_final.astype(np.uint8)

In [ ]:
path = 'dataset/meanshift/predictions/positivo/final/'

#create path
import os
if not os.path.exists(path):
    os.makedirs(path)
    
plt.imshow(img_final, cmap='gray')

In [ ]:
import numpy as np
import tensorflow as tf
from skimage.transform import resize

def preprocess_image(image, target_size=(128, 128)):
    # Redimensiona a imagem para o tamanho alvo
    image = resize(image, target_size, anti_aliasing=True)

    # Normaliza a imagem (exemplo: escala de 0 a 1)
    image = image / 255.0

    return image

# Preprocessa todas as imagens originais e segmentadas
processed_orig_images = np.array([preprocess_image(img) for img in dataset['img']])
processed_segmented_images = np.array([preprocess_image(img) for img in imagens_meanshift2])

In [ ]:
# Verifica se as imagens estão em escala de cinza e as expande para terem 3 canais, se necessário
if processed_orig_images.ndim < 4:
    processed_orig_images = np.stack((processed_orig_images,)*3, axis=-1)

if processed_segmented_images.ndim < 4:
    processed_segmented_images = np.stack((processed_segmented_images,)*3, axis=-1)

# Combina as imagens
combined_images = np.concatenate((processed_orig_images, processed_segmented_images), axis=-1)


In [ ]:
plt.imshow(dataset['mask'][2], cmap='gray')

In [ ]:
from plots import funcaoPlot

funcaoPlot([dataset['img'][0], dataset['mask'][0], imagens_meanshift2[0]])

In [ ]:
funcaoPlot([dataset['img'][200], dataset['mask'][200], imagens_meanshift2[200]])

In [ ]:
funcaoPlot([dataset['img'][2000], dataset['mask'][2000], imagens_meanshift2[2000]])

In [ ]:
funcaoPlot([dataset['img'][20000], dataset['mask'][20000], imagens_meanshift2[20000]])